In [ ]:
from __future__ import print_function, division
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
import os
import glob
%matplotlib inline

###1. Read the station data downloaded from GHCN archive###
Data obtained from http://www.ncdc.noaa.gov/cdo-web/

In [ ]:
def get_date(date_number):
    """
    Turn the int64 value from the DATE of GHCN into a pd.datetime
    """
    dstring = str(date_number)
    return pd.datetime(int(dstring[0:4]),int(dstring[4:6]),int(dstring[6:8]))

def get_df(fnm, var, no_missing = True):
    """
    Create a dataframe for a single station, with a time index, for a single
    variable of data given as a key word (e.g. PRECIP, TMAX, TMIN).
    Requires file path and name (fnm).
    no_missing is a Bool that optionally masks out values < -99 from the df.
    """
    df = pd.read_csv(fnm)
    dt_indx = [get_date(date) for date in df.DATE]
    data_vals = df[var].values
    if var is 'PRCP':
        data_vals = data_vals / 10.  # This is to convert precip data to mm
    if no_missing:
        tmp_df = pd.DataFrame(data=data_vals,
                              index=dt_indx,columns=[df.STATION[0][6:]])
        mask = tmp_df > -99.  # A catchall value for missing data in GHCN
        return tmp_df[mask]
    else:
        return pd.DataFrame(data=data_vals,
                            index=dt_indx,columns=[df.STATION[0][6:]])

def get_combined_df(fpth, var):
    """
    From a given file path, and variable, extract data from all .csv files, and
    place in a single dataframe object.
    """
    flist = glob.glob(fpth)
    df_dic = {}
    for f in flist:
        df_dic[f[5:]] = get_df(fnm = f, var = var, no_missing=True)
    return pd.concat([df_dic[key] for key in df_dic.keys()],axis=1)

Call the Get_combined() function to create dataframes out of all data in a folder.

In [ ]:
%%time
df_tmax = get_combined_df(fpth="Data/*.csv",var="TMAX")
df_tmin = get_combined_df(fpth="Data/*.csv",var="TMIN")
df_prcp = get_combined_df(fpth="Data/*.csv",var="PRCP")

In [ ]:
for station in df_prcp:
    print(station, np.max(df_prcp[station]))

Plot time series of precipitation for all stations, and also accumulate the data and plot the average rainfall.

In [ ]:
# Example of masking and accessing data from stations...
#station = df_prcp.keys()[4]
#plt.plot(df_prcp[station].index,df_prcp[station],'.',alpha=0.5)
#plt.title("Station {0:s}".format(station))

In [ ]:
#df_prcp.KE000063740[df_prcp.KE000063740 > 100]

###2. Time series plots###

Daily Mean and SEM values: Mean uncertainty is given by SEM, where:
SEM=σn

In [ ]:
def calc_SEM(data):
    """
    Calculate Standard error of the mean. No nan's 
    should be in the input (numpy) array.
    """
    return np.std(data)/np.sqrt(len(data) - 1)


def gather_daily_stats(date, df):
    """
    For a specified day, given by date, create a short array of 
    observed values (obs) excluding the NANs. Return the mean, 
    and SEM value.
    Restrictions: more than one observation on a day, not a missing
    value, less than 300 mm per day (which is erroneous).
    """
    obs = np.array([df_prcp[key][day] for key in df_prcp.keys()])
    obs = obs[(obs > -1) & (obs < 300)]
    if len(obs) < 2:
        return np.NAN, np.NAN
    return np.mean(obs), calc_SEM(obs)


In [ ]:
#MAD based outlier calculation.
#def rej_Olier(data, thresh = 0.):
#    """
#    Calculate biweights of mean to reject outliers in df_prcp. No nan's 
#    should  also be in the input (numpy) array.
#    """
#    diff = np.abs(data - np.median(data))
#    mad = np.median(diff)   #median of the absolute deviation
#    mod_obs = diff/mad if mad else 0.
#    return data[mod_obs > thresh]

In [ ]:
# Create an accumulated time series (with SEM uncertainty values)
means = []
sems = []
for day in df_prcp.index:
    tmp_mean, tmp_sem = gather_daily_stats(date=day, df=df_prcp)
    means.append(tmp_mean)
    sems.append(tmp_sem)
means = np.array(means)
sems = np.array(sems)
df_prcp['Accumulated']=pd.Series(means,index=df_prcp.index)  #adding columns to the dataframe!
df_prcp['Acc_SEM']=pd.Series(sems,index=df_prcp.index)

In [ ]:
daily_ts = plt.figure(dpi=72)
daily_ts.set_size_inches(10,5)      # Specify the figure size
ax1 = daily_ts.add_subplot(111)     # Add an axis frame object to the plot (i.e. a pannel)

ax1.errorbar(df_prcp.index, df_prcp.Accumulated,
             yerr=df_prcp.Acc_SEM, alpha=0.75, fmt=',')
ax1.set_ylim(0,120)
plt.xlim('1950-01-01','2015-08-31')
plt.title("Mean East African Precipitation")
plt.ylabel("Precipitation (mm day$^{-1}$)")
plt.xlabel("Year")
plt.grid(True)
plt.show(daily_ts)
#daily_ts.savefig('Daily_ts.pdf',dpi=300)

###3. Density plots###

In [ ]:
# N.b. the KDE (kernel density estimate) is Gaussian - which is not true
# for precip data (log or power law data)...
mask = df_prcp.Accumulated > 0.0
daily_dp = plt.figure()
daily_dp.set_size_inches(10,5)
ax = daily_dp.add_subplot(122)

sns.distplot(df_prcp.Accumulated[mask],bins=1000, norm_hist = True,
             kde=False,color = 'r')
sns.kdeplot(df_prcp.Accumulated[mask],shade=True,kernel='cos',cumulative=False,color='b')
ax.set_xlim(-10,50)
ax.set_title("East Africa Accumulated mean")
ax.set_xlabel(r'Precip. (mm day$^{-1}$)')
ax.set_ylabel('Density (0-1)')

plt.show(daily_ts)

In [ ]:
sns.kdeplot # Hot tip - look in SEABORN for statistical plots and help...

Tasks:
1. find out why the later part of the data has high variability
2. make sure you are happy/add any logical restrictions to improve the data quality in Accumulated dataset
3. Caclulate population statistics, histrogram, density plots (PDF, CDF), and fits to the population. Try several fit approaches, and show which is best.
4. Use the CDF (or a percentile function) to determine the key (IQR, median, tails etc) of the population
5. (hard) try to fit to the population. Reccomend trying a nth order polyfit using np.polyfit()
6. Use the statistical threshold values to define 'extreme' precipitation, and work out the:
  * frequency of extreme events,
  * duration (lenght) of extreme events,
  * magnitude (intensity) of extreme events
  
For task 6, you can plot these statistics as time dependent, or distributions, or something else...

In [ ]:
#plt.hist(df_prcp.Accumulated[mask], bins=60)
#plt.show()

In [ ]:
#Histogram
hist_dp = plt.figure()
hist_dp.set_size_inches(5,5)        # Specify the output size
ax1 = hist_dp.add_subplot(211)        # Add an axis frame object to the plot (i.e. a pannel)
ax2 = hist_dp.add_subplot(212)        # Add an axis frame object to the plot (i.e. a pannel)

# the histogram of the data
ax1.set_title(r' Mean East African Precipitation')
n, bins, patches = ax1.hist(df_prcp.Accumulated[mask], 60, normed=True, facecolor='blue', alpha=0.75,
                            histtype='stepfilled')
ax1.grid(True)
ax1.set_ylabel('Density')
n, bins, patches = ax2.hist(df_prcp.Accumulated[mask], 60, normed=True, facecolor='blue', alpha=0.75,
                            histtype='stepfilled',cumulative=True)
plt.xlabel(r'mm day$^{-1}$')
plt.ylabel('Cumulative density')
plt.grid(True)

plt.show()
#my_hist.savefig('Density_plots.pdf',dpi=300)

In [ ]:
np.max(df_prcp.Accumulated)

###4. Seasonality###

Calculate the DOY mean over the data-period (climatology).

In [ ]:
doy_mean=[]
doy_sem =[]
doy_csum=[]
for doy in range(366):
    index = df_prcp.index.dayofyear == doy+1 
    doy_mean.append(np.nanmean(df_prcp[index]))
    doy_sem.append(np.nanstd(df_prcp[index])/np.sqrt(len(df_prcp[index])-1))  

In [ ]:
plt.errorbar(range(366),doy_mean,xerr=None, yerr=doy_sem, alpha=0.6)
plt.xlim(0,max(range(366)))
plt.title("East Africa DOY Mean ($\mu$) Rainfall")
plt.ylabel("Precipitation (mm)")
plt.xlabel("Day of Year")

Anomaly

Use the seasonal DOY mean to calculate deviations (anomaly) from the daily mean

In [ ]:
##Anomalies = Observation - Climatology
anomalies = []
for n,day in enumerate(df_prcp.index):
    #print("Index: {0} Date: {1} Value: {0:3.3f}".format(n,df_prcp.index[n].date(),day[0]))
    doyi = df_prcp.index[n].dayofyear -1 # Create an index to call doy_mean
    #print(n, doy_mean[doyi])
    #anomalies.append(day - doy_mean[doyi])


Correlation Matrix

In [ ]:
corr_matrix = df_prcp
corr_matrix.corr(method='pearson', min_periods=1)

calculate the percentiles

In [ ]:
def percentile(df, percentile):
   # size = len(df_prcp)
     
    p90 = []
    p95 = []
    p99 = [] 
    for day in df_prcp.index:
        tmp = np.array([df_prcp[key][day] for key in df_prcp.keys()])
        p90.append(np.percentile(tmp,90))    # ...find the nth percentile uncertainty values 
        p95.append(np.percentile(tmp,95)) 
        p99.append(np.percentile(tmp,99)) 
        
        
        return p90, p95, p99

In [ ]:
#from scipy.stats import scoreatpercentile
# percentiles of interest
#tmp = np.array([df_prcp[key][day] for key in df_prcp.keys()])
#perc = [min(tmp), scoreatpercentile(tmp,10), scoreatpercentile(tmp,25),
              # scoreatpercentile(tmp,50), scoreatpercentile(tmp,75),
               #scoreatpercentile(tmp,90), max(tmp)]